In [ ]:
"""
セグメント木


配列にたいして、
・区間[a,b)について最小やら最大やらを求める
・要素iの値を変更する
を高速に行える

詳しくは：https://www.slideshare.net/iwiwi/ss-3578491?ref=https://juppy.hatenablog.com/entry/2019/05/02/%E8%9F%BB%E6%9C%AC_python_%E3%82%BB%E3%82%B0%E3%83%A1%E3%83%B3%E3%83%88%E6%9C%A8_%E7%AB%B6%E6%8A%80%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0_Atcoder


実装についてはここを見るとわかりやすい
http://tsutaj.hatenablog.com/entry/2017/03/29/204841


下は一例なので必要に合わせて都度修正
たとえばたこ焼きの問題のqueryみたいにretl,retrがあるパターンや、累積gcdの問題みたいにretだけでいいやつとか
（gcdはどの順番でとっても結果が変わらないので、retだけでよいと認識している）

とりあえず下の汎用性の高い方を使うようにする
"""

class SegmentTree:
    # セグメント木として初期化する配列、単位元、[a,b)に実行する関数
    def __init__(self, lst, ide_ele, segfunc):
        self.ide_ele = ide_ele
        self.func = segfunc
        # lstのサイズより大きい2^xで最小の値
        self.size = 2**(len(lst) - 1).bit_length()
        self.seg_lst = [self.ide_ele] * 2 * self.size

        """
        値を詰めていく
        ノードiの親：(i-1) // 2
        ノードiの子：2*i+1 と　2*i+2

        """
        for i in range(len(lst)):
            self.seg_lst[i + self.size - 1] = lst[i]

        # クエリが実行されたときに取り出せるように処理をしておく
        # この辺は何がしたいかによって自分で関数用意する
        for i in range(self.size-2, -1, -1):
            # 子から親に向かって結果を吸い上げるような形
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def update(self, i, num):
        # 一番下の子の部分を更新するため、配列中のindexからseg_lstのindexに変える
        i += self.size - 1
        self.seg_lst[i] = num
        while i:
            # 親に更新を伝播
            i = (i - 1) // 2
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def query(self, l, r):
        # 範囲がおかしい場合
        if r <= l:
            return self.ide_ele

        # seg_lst内でのindexに直す
        l += self.size - 1
        # 
        r += self.size - 2
        ret = self.ide_ele
        # 区間が閉じ切る = 同じ親のところに戻るまで
        while r - l > 1:
            # lの下位1ビットが立っていない。たぶんだが、あるノードの左右の子の左が範囲に入ってるかどうかの判断で、入ってないときか？
            if l & 1 == 0:
                # 一番初めは単位減であるretとlについてfuncをかけることになるので、やはり左の子が範囲に入ってないとき！
                ret = self.func(ret, self.seg_lst[l])
            # こっちは右の子が入ってるかどうかで、入っている場合の処理だと思う
            if r & 1 == 1:
                # 一番最初は単位元とrの処理なので、右側が範囲に入ってない時。
                ret = self.func(ret, self.seg_lst[r])
                r -= 1

            # それぞれの親に移動というか、一個上のdepthに移動
            l = l // 2
            r = (r - 1) // 2

        # retには下から積み上げてきたself.funcの結果が入ってるので、たどり着いた親の所で同じく処理を実行して戻す
        if l == r:
            ret = self.func(ret, self.seg_lst[l])
        else:
            ret = self.func(self.func(ret, self.seg_lst[l]), self.seg_lst[r])
        
        return ret

In [ ]:
"""
汎用性高めのセグメント木はこっちの方で

"""

class SegmentTree():

    def __init__(self, init_list, ide_ele, seg_func):
        self.N = 2**(len(init_list) - 1).bit_length()
        self.e = ide_ele
        self.f = seg_func
        self.tree = [self.e for _ in range(self.N * 2)]

        # 初期状態を木の最下層に追加。最下層より上のノードの個数がN-1なので、+(N-1)している
        for i in range(len(init_list)):
            self.tree[i + self.N - 1] = init_list[i]

        # 子のノードから情報をくみ上げる
        for i in range(self.N - 2, -1, -1):
            # ノードiの子は2*i+1と2*i+2
            self.tree[i] = self.f(self.tree[2*i + 1], self.tree[2*i + 1])

    # [l,r)の範囲でseg_funcにかけた物を返す。
    def update(self, i, x):
        # 木の最下層のインデックスに変える
        i += self.N - 1
        self.tree[i] = x
        # 更新内容を親に反映
        while i:
            # ノードiの親の番号がこれ
            i = (i - 1) // 2
            self.tree[i] = self.f(self.tree[2*i + 1],self.tree[2*i + 2])

    def query(self, l, r):
        if r <= l:
            return self.e
        # 木の最下層における番号に変える
        l += self.N - 1
        # rは開区間で渡されるから、配列の要素数に収まるように、-2となっている
        r += self.N - 2

        # 根の左右それぞれについての、区間が子の一方のみしかカバーしない場合に使う値
        ret_l = self.e
        ret_r = self.e

        # 区間が収束するまで。二分探索と同じノリ
        while r - l > 1:
            # lの下位１ビットが立っていない = 偶数 = 左の部分木のあるノードの左の子が区間に含まれない場合
            # 詳しくはセグメント木を書いて配列のindexをはめていけばわかる。
            if l & 1 == 0:
                ret_l = self.f(ret_l, self.tree[l])
            
            # 右の部分木のノードの右の子が範囲に含まれない場合
            # 詳しくはセグメント木を書いて配列のindexをはめていけばわかる。
            if r & 1 == 1:
                ret_r = self.f(self.tree[r], ret_r)
                # 右側の区間を一個左につめて、左右の子が範囲に収まるようにした感じ
                r -= 1

            # それぞれの親に移動
            # この操作も木を書いて配列のindexを貼れば分かる
            l = l // 2
            r = (r - 1) // 2

        # それぞれで親に移っていったときに、同じ親（根以外）で合流したとき = 根の左右の部分木の一方にl,rの範囲が指定されていたとき
        if l == r:
            ret_l = self.f(ret_l, self.tree[l])
        # 根で合流したとき
        else:
            # ここは｛ ret_l, tree[l], tree[r], ret_r ｝みたいなのをくみ上げるための処理
            ret_l = self.f(ret_l, self.tree[l])
            ret_l = self.f(ret_l, self.tree[r])
        
        return self.f(ret_l, ret_r)

### セグメント木ここまで ###


In [ ]:
"""
AtCoder Beginner Contest 125  C - GCD on Blackboard

配列中の数字を一つ好きな値に書き換えて最大公約数を計算するときの最大値を求める
・書き換える→取り除くと考えてよい。
・全体のGCD = GCD(前半のGCD, 後半のGCD) のように、分割したり順番を入れ替えたりしても結果は変わらない。
なので、左からの累積GCDと、右からの累積GCDをあらかじめとっておくことで、要素を一個除いたときの全体のGCDも求められる。　

区間に対する処理 -> セグメント木いけるやん
"""
class SegmentTree:
    # セグメント木として初期化する配列、単位元、[a,b)に実行する関数
    def __init__(self, lst, ide_ele, segfunc):
        self.ide_ele = ide_ele
        self.func = segfunc
        # lstのサイズより大きい2^xで最小の値
        self.size = 2**(len(lst) - 1).bit_length()
        self.seg_lst = [self.ide_ele] * 2 * self.size

        """
        値を詰めていく
        ノードiの親：(i-1) // 2
        ノードiの子：2*i+1 と　2*i+2

        """
        for i in range(len(lst)):
            self.seg_lst[i + self.size - 1] = lst[i]

        # クエリが実行されたときに取り出せるように処理をしておく
        # この辺は何がしたいかによって自分で関数用意する
        for i in range(self.size-2, -1, -1):
            # 子から親に向かって結果を吸い上げるような形
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def update(self, i, num):
        # 一番下の子の部分を更新するため、配列中のindexからseg_lstのindexに変える
        i += self.size - 1
        self.seg_lst[i] = num
        while i:
            # 親に更新を伝播
            i = (i - 1) // 2
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def query(self, l, r):
        # 範囲がおかしい場合
        if r <= l:
            return self.ide_ele

        # seg_lst内でのindexに直す
        l += self.size - 1
        # 
        r += self.size - 2
        ret = self.ide_ele
        # 区間が閉じ切る = 同じ親のところに戻るまで
        while r - l > 1:
            # lの下位1ビットが立っていない。たぶんだが、あるノードの左右の子の左が範囲に入ってるかどうかの判断で、入ってないときか？
            if l & 1 == 0:
                # 一番初めは単位減であるretとlについてfuncをかけることになるので、やはり左の子が範囲に入ってないとき！
                ret = self.func(ret, self.seg_lst[l])
            # こっちは右の子が入ってるかどうかで、入っている場合の処理だと思う
            if r & 1 == 1:
                # 一番最初は単位元とrの処理なので、右側が範囲に入ってない時。
                ret = self.func(ret, self.seg_lst[r])
                r -= 1

            # それぞれの親に移動というか、一個上のdepthに移動
            l = l // 2
            r = (r - 1) // 2

        # retには下から積み上げてきたself.funcの結果が入ってるので、たどり着いた親の所で同じく処理を実行して戻す
        if l == r:
            ret = self.func(ret, self.seg_lst[l])
        else:
            ret = self.func(self.func(ret, self.seg_lst[l]), self.seg_lst[r])
        
        return ret

    
from fractions import gcd
N = int(input())
A = list(map(int, input().split()))

seg_tree = SegmentTree(A, 0, gcd)

ans = -1

for i in range(N):
    # 要素iを除いたときの全体のGCD
    ans = max(ans, gcd(seg_tree.query(0, i), seg_tree.query(i+1, N)))

print(ans)

In [ ]:
"""
AOJ Course Range Minimum Query (RMQ)


普通のセグメント木
クラスにする意味があまり感じられないので、ふつうに書く練習として。

"""

N,Q = map(int, input().split())

A = [2**31 - 1] * N

q = [tuple(map(int, input().split())) for _ in range(Q)]

seg_num = 2**(N-1).bit_length()
ide_ele = float("inf")
func = min
seg_lst = [ide_ele] * 2 * seg_num

def init(lst):
    for i in range(N):
        # seg_lst1内のindexに変換
        seg_lst[i + seg_num - 1] = lst[i]
    
    # それぞれのノードについて、子の部分を引き上げてくる
    for i in range(seg_num - 2, -1, -1):
        seg_lst[i] = func(seg_lst[2*i + 1], seg_lst[2*i + 2])

def update(i, x):
    # seg_lst1内のindexに変換
    i += seg_num - 1
    seg_lst[i] = x
    while i:
        # 親に移って子の部分をくみ上げる
        i = (i - 1) // 2
        seg_lst[i] = func(seg_lst[2*i + 1], seg_lst[2*i + 2])

def query(l,r):
    if r <= l:
        return ide_ele

    l += seg_num - 1
    r += seg_num - 2

    ret = ide_ele

    while r - l > 1:
        # 左が範囲に含まれておらず単位元扱いの場合
        if l & 1 == 0:
            ret = func(ret, seg_lst[l])
        # 右が範囲に含まれておらず単位元扱いの場合
        if r & 1 == 1:
            ret = func(ret, seg_lst[r])
            r -= 1
        l  = l // 2
        r = (r - 1) // 2

    if l == r:
        ret = func(ret, seg_lst[l])
    else:
        ret = func(ret, seg_lst[l])
        ret = func(ret, seg_lst[r])
    return ret


init(A)

for c,x,y in q:
    if c == 0:
        update(x, y)
    else:
        print(query(x, y+1))

In [ ]:
"""

AtCoder Regular Contest 008  D - タコヤキオイシクナール

セグメント木の練習

x -> a1*x+b1 -> a2*(a1*x+b1) + b2 -> ...
みたいな形で出てくるので、セグメント木で考えた時に、箱1と箱2の結果を、その上のノードが持つ形になる。
上まですい上げていくと、一番上に全ての箱を通った時のたこ焼きの美味しさを出すためのax+bのaとb（うまく言えないが、全ての箱をまとめて一つの箱だと考えた時のaとb）が出てくる
単位元は、たこ焼きが箱を通っても何も起こらない場合なので、a=1,b=0となる。

Nの範囲が大きすぎて配列で持つとTLEになるが、実際の所、クエリが実行されるM要素以外は存在しないと考えていい（単位元なので結果に影響しない）
そのため、Mまで座標圧縮する
"""
class SegmentTree():

    def __init__(self, init_list, ide_ele, seg_func):
        self.N = 2**(len(init_list) - 1).bit_length()
        self.e = ide_ele
        self.f = seg_func
        self.tree = [self.e for _ in range(self.N * 2)]

        # 初期状態を木の最下層に追加。最下層より上のノードの個数がN-1なので、+(N-1)している
        for i in range(len(init_list)):
            self.tree[i + self.N - 1] = init_list[i]

        # 子のノードから情報をくみ上げる
        for i in range(self.N - 1)[::-1]:
            # ノードiの子は2*i+1と2*i+2
            self.tree[i] = self.f(self.tree[2*i + 1], self.tree[2*i + 1])

    # [l,r)の範囲でseg_funcにかけた物を返す。
    def update(self, i, x):
        # 木の最下層のインデックスに変える
        i += self.N - 1
        self.tree[i] = x
        # 更新内容を親に反映
        while i:
            # ノードiの親の番号がこれ
            i = (i - 1) // 2
            self.tree[i] = self.f(self.tree[2*i + 1],self.tree[2*i + 2])

    def query(self, l, r):
        if r <= l:
            return self.e
        # 木の最下層における番号に変える
        l += self.N - 1
        # rは開区間で渡されるから、配列の要素数に収まるように、-2となっている
        r += self.N - 2

        # 根の左右それぞれについての、区間が子の一方のみしかカバーしない場合に使う値
        ret_l = self.e
        ret_r = self.e

        # 区間が収束するまで。二分探索と同じノリ
        while r - l > 1:
            # lの下位１ビットが立っていない = 偶数 = 左の部分木のあるノードの左の子が区間に含まれない場合
            # 詳しくはセグメント木を書いて配列のindexをはめていけばわかる。
            if l & 1 == 0:
                ret_l = self.f(ret_l, self.tree[l])
            
            # 右の部分木のノードの右の子が範囲に含まれない場合
            # 詳しくはセグメント木を書いて配列のindexをはめていけばわかる。
            if r & 1 == 1:
                ret_r = self.f(self.tree[r], ret_r)
                # 右側の区間を一個左につめて、左右の子が範囲に収まるようにした感じ
                r -= 1

            # それぞれの親に移動
            # この操作も木を書いて配列のindexを貼れば分かる
            l = l // 2
            r = (r - 1) // 2

        # それぞれで親に移っていったときに、同じ親（根以外）で合流したとき = 根の左右の部分木の一方にl,rの範囲が指定されていたとき
        if l == r:
            ret_l = self.f(ret_l, self.tree[l])
        # 根で合流したとき
        else:
            # ここは｛ ret_l, tree[l], tree[r], ret_r ｝みたいなのをくみ上げるための処理
            ret_l = self.f(ret_l, self.tree[l])
            ret_l = self.f(ret_l, self.tree[r])
        
        return self.f(ret_l, ret_r)

### セグメント木ここまで ###

N,M = map(int, input().split())

PAB = [[f(x) for f, x in zip([int, float, float], input().split())] for _ in range(M)]

# Nの範囲が10**12と大きすぎるが、実際に必要なのはM種だけなので座標圧縮
P_to_idx = {p:i for i,p in enumerate(sorted(set(p for p, _, _ in PAB)))}

ide_ele = (1.0, 0.0)
lst = [ide_ele] * len(P_to_idx.keys())

# タコヤキオイシクナールを複数かけていくときの合成関数
def seg_func(x,y):
    return (x[0]*y[0], x[1]*y[0] + y[1])

seg_tree = SegmentTree(lst, ide_ele, seg_func)

ans_min = 1.0
ans_max = 1.0

for p,a,b in PAB:
    seg_tree.update(P_to_idx[p], (a,b))
    tmp = sum(seg_tree.query(0, len(P_to_idx.keys())))
    ans_max = max(ans_max, tmp)
    ans_min = min(ans_min, tmp)


print(ans_min)
print(ans_max)

Error: Jupyter server crashed. Unable to connect. 
Error code from jupyter: 1

In [ ]:
"""






"""

In [ ]:
"""

"""